In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

ROBOFLOW_API_KEY = os.environ.get('ROBOFLOW_API_KEY')

# 1. 데이터셋 불러오기

In [9]:
from roboflow import Roboflow

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace("hayley-kim-6vkvh").project("chiikawa-doll-rebbit-smallba-cjhim")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Chiikawa-doll-rebbit-smallba--1 in yolov11:: 100%|██████████| 984/984 [00:00<00:00, 2288.14it/s]


# 2. 모델 불러오기

In [12]:
data_path = "C:\Potenup\DeepLearning-YOLO-Study\src\Chiikawa-doll-rebbit-smallba--1\data.yaml"
model_path = "C:\Potenup\DeepLearning-YOLO-Study\models\yolo11n.pt"

In [13]:
from ultralytics import YOLO

# model = YOLO(data_path) # 새로운 데이터로 완전히 다시 학습
model = YOLO(model_path) # YOLO 의 모델을 그대로 사용하고, YOLO의 가중치를 사용, 내 데이터로 파인튜닝(가중치 업데이트)
# model = YOLO(data_path).load(model_path) # YOLO의 가중치를 가져와서, 새로 만든 모델에 적용

# 3. 학습하기

In [17]:
results = model.train(data=data_path, epochs=10, imgsz=640, save=True)

Ultralytics 8.3.191  Python-3.10.18 torch-2.8.0+cpu CPU (Intel Core(TM) Ultra 9 185H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Potenup\DeepLearning-YOLO-Study\src\Chiikawa-doll-rebbit-smallba--1\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Potenup\DeepLearning-YOLO-Study\models\yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train134, nbs=64, nms=False, opset=None, optimize=False, 

# 4. 평가하기

## 최고 성능 모델 불러오기

In [18]:
load_model = YOLO("C:\Potenup\DeepLearning-YOLO-Study\\runs\detect\\train134\weights\\best.pt")

In [19]:
val_results = load_model.val(data=data_path, imgsz=640, iou=0.5, save=True)
# conf(신뢰도 임계값) : 모델이 객체를 탐지했다고 판단하는 최소한의 신뢰도 값
# iou(IoU (Intersection over Union) Threshold) : 탐지된 박스와 실제 정답 박스(Ground Truth)가 얼마나 겹치는지를 판단하는 임계값

Ultralytics 8.3.191  Python-3.10.18 torch-2.8.0+cpu CPU (Intel Core(TM) Ultra 9 185H)
YOLO11n summary (fused): 100 layers, 2,584,687 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 402.0327.8 MB/s, size: 37.5 KB)
val: Scanning C:\Potenup\DeepLearning-YOLO-Study\src\Chiikawa-doll-rebbit-smallba--1\valid\labels.cache... 97 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 97/97  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.7it/s 4.1s0.7sss
                   all         97        424      0.861      0.556      0.652      0.406
                3dolls          2          2          1          0     0.0831     0.0332
             Sea_otter          6          9      0.916      0.444      0.523      0.286
                 Shisa          9          9      0.756      0.667        0.7      0.609
                 anoko          3          3       0.88          1      0.995       0.41
 

# 5. 사용하기

In [26]:
predict_results = load_model("C:\Potenup\DeepLearning-YOLO-Study\images\chiikawa_test.jpeg", save=True)


image 1/1 C:\Potenup\DeepLearning-YOLO-Study\images\chiikawa_test.jpeg: 640x640 1 chiikawa, 1 kurima, 1 momo, 2 small bas, 1 usaki, 71.3ms
Speed: 3.0ms preprocess, 71.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Potenup\DeepLearning-YOLO-Study\runs\detect\predict5


In [27]:
predict_results[0].show()

In [25]:
class_names = predict_results[0].names

for cls in predict_results[0].boxes.cls:
    print(class_names[cls.item()])

small ba
kurima
momo
usaki
small ba
chiikawa
